In [6]:
import subprocess

# This is a sample Python/Flask app showing Domino's App publishing functionality.
# You can publish an app by clicking on "Publish" and selecting "App" in your
# quick-start project.

import json
import flask
from flask import request, redirect, url_for
import numpy as np
import logging
import requests
import sys
import time

class ReverseProxied(object):
  def __init__(self, app):
      self.app = app
  def __call__(self, environ, start_response):
      script_name = environ.get('HTTP_X_SCRIPT_NAME', '')
      if script_name:
          environ['SCRIPT_NAME'] = script_name
          path_info = environ['PATH_INFO']
          if path_info.startswith(script_name):
              environ['PATH_INFO'] = path_info[len(script_name):]
      # Setting wsgi.url_scheme from Headers set by proxy before app
      scheme = environ.get('HTTP_X_SCHEME', 'https')
      if scheme:
        environ['wsgi.url_scheme'] = scheme
      # Setting HTTP_HOST from Headers set by proxy before app
      remote_host = environ.get('HTTP_X_FORWARDED_HOST', '')
      remote_port = environ.get('HTTP_X_FORWARDED_PORT', '')
      if remote_host and remote_port:
          environ['HTTP_HOST'] = f'{remote_host}:{remote_port}'
      return self.app(environ, start_response)

app = flask.Flask(__name__)
#app.wsgi_app = ReverseProxied(app.wsgi_app)

# Homepage which uses a template file
@app.route('/')
def index_page():
  return flask.render_template("index.html")

@app.route('/', methods=['POST'])
def my_form_post():
    age = 1 #request.form['age']
    print('age', age, ' : ', type(age))
    sex = 1 #request.form['sex']
    print('sex', sex, ' : ', type(sex))
    cp = 1 #request.form['cp']
    trtbps = 1 #request.form['trtbps']
    chol = 1 #request.form['chol']
    fbs = 1 #request.form['fbs']
    restecg = 1 #request.form['restecg']
    thalachh = 1 #request.form['thalachh']

    exng = 1 #request.form['exng']
    oldpeak = 1 #request.form['oldpeak']
    slp = 1 #request.form['slp']

    caa = 1 #request.form['caa']
    thall = 1 #request.form['thall']

    data = {
        "age": float(age),
        "sex": float(sex),
        "cp": float(cp),
        "trtbps": float(trtbps),
        "chol": float(chol),
        "fbs": float(fbs),
        "restecg": float(restecg),
        "thalachh": float(thalachh),
        "exng": float(exng),
        "oldpeak": oldpeak,
        "slp": float(slp),
        "caa": float(caa),
        "thall": float(thall)
    }


    # change logging setup as required
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)  

    # TO EDIT: update the example request parameters for your model
    REQUEST_PARAMETERS = data
    
    # TO EDIT: copy these values from "Calling your Model" on the Model API overview page
    DOMINO_URL = "https://cdc-sandbox.domino-eval.com:443"
    MODEL_ID = "6536d231f66ed97e65981017"
    MODEL_ACCESS_TOKEN = "vPukaQwUJwv04z8S2VGDspZLtVM8575asNbGORb6VjYU6JLhvEGxZLjp3YGMKORk"
 
    # DO NOT EDIT these values
    MODEL_BASE_URL = f"{DOMINO_URL}/api/modelApis/async/v1/{MODEL_ID}"
    SUCCEEDED_STATUS = "succeeded"
    FAILED_STATUS = "failed"
    QUEUED_STATUS = "queued"
    TERMINAL_STATUSES = [SUCCEEDED_STATUS, FAILED_STATUS]
    PENDING_STATUSES = [QUEUED_STATUS]
    MAX_RETRY_DELAY_SEC = 60


    ### CREATE REQUEST ###

    create_response = None
    retry_delay_sec = 0
    while (
        create_response is None
        or (500 <= create_response.status_code < 600)  # retry for transient 5xx errors
    ):
        # status polling with a time interval that backs off up to MAX_RETRY_DELAY_SEC
        if retry_delay_sec > 0:
            time.sleep(retry_delay_sec)
        retry_delay_sec = min(max(retry_delay_sec * 2, 1), MAX_RETRY_DELAY_SEC)
        
        create_response = requests.post(
            MODEL_BASE_URL,
            headers={"Authorization": f"Bearer {MODEL_ACCESS_TOKEN}"},
            json={"parameters": REQUEST_PARAMETERS}
        )

    if create_response.status_code != 200:
        raise Exception(f"create prediction request failed, response: {create_response}")

    prediction_id = create_response.json()["asyncPredictionId"]
    logging.info(f"prediction id: {prediction_id}")


    ### POLL STATUS AND RETRIEVE RESULT ###

    status_response = None
    retry_delay_sec = 0
    while (
            status_response is None
            or (500 <= status_response.status_code < 600)  # retry for transient 5xx errors
            or (status_response.status_code == 200 and status_response.json()["status"] in PENDING_STATUSES)
    ):
        # status polling with a time interval that backs off up to MAX_RETRY_DELAY_SEC
        if retry_delay_sec > 0:
            time.sleep(retry_delay_sec)
        retry_delay_sec = min(max(retry_delay_sec * 2, 1), MAX_RETRY_DELAY_SEC)

        status_response = requests.get(
            f"{MODEL_BASE_URL}/{prediction_id}",
            headers={"Authorization": f"Bearer {MODEL_ACCESS_TOKEN}"},
        )

    if status_response.status_code != 200:
        raise Exception(f"prediction status request failed, response: {create_response}")

    prediction_status = status_response.json()["status"]
    if prediction_status == SUCCEEDED_STATUS:  # succeeded response includes the prediction result in "result"
        result = status_response.json()["result"]
        logging.info(f"prediction succeeded, result:\n{json.dumps(result, indent = 2)}")
    elif prediction_status == FAILED_STATUS:  # failed response includes the error messages in "errors"
        errors = status_response.json()["errors"]
        logging.error(f"prediction failed, errors:\n{json.dumps(errors, indent = 2)}")
    else:
        raise Exception(f"unexpected terminal prediction response status: {prediction_status}") 
    print(response.status_code)
    print(response.headers)
    print(response.json())

    return response.json()




In [9]:
#y_form_post()

In [49]:
data = {"age": 1, "sex": 1, "cp":1,"trtbps": 1,"chol": 1,"fbs": 1,"restecg": 1,
        "thalachh": 1, "exng": 1, "oldpeak": "1", "slp": 1, "caa": 1, "thall": 1}

DOMINO_URL = 'https://cdc-sandbox.domino-eval.com'
MODEL_ID = '6536d231f66ed97e65981017'
MODEL_ACCESS_TOKEN = 'CueoJUbSdWoGigkCCQNNfoa89eC8f0xZKuS0LUNyL3ijayyhSIj3HL4Qulvwmbtx'
AUTH=("CueoJUbSdWoGigkCCQNNfoa89eC8f0xZKuS0LUNyL3ijayyhSIj3HL4Qulvwmbtx", "CueoJUbSdWoGigkCCQNNfoa89eC8f0xZKuS0LUNyL3ijayyhSIj3HL4Qulvwmbtx")
AUTH=(f"{MODEL_ACCESS_TOKEN}", f"{MODEL_ACCESS_TOKEN}")

response = requests.post(f"{DOMINO_URL}/models/{MODEL_ID}/latest/model",
   auth=AUTH,
   json={"data": data}
)

print(response.status_code)
print(response.headers)
print(response.json())

200
{'Date': 'Tue, 24 Oct 2023 18:40:20 GMT', 'Content-Type': 'application/json', 'Content-Length': '227', 'Connection': 'keep-alive', 'X-Request-ID': 'R7N5VHD1ZP3QOUSA', 'Domino-Server': 'nginx-ingress,model-api,', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'POST', 'Access-Control-Allow-Headers': 'authorization,content-type', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains'}
{'model_time_in_ms': 16, 'release': {'harness_version': '0.1', 'registered_model_name': 'lgb_heart_attack_model', 'registered_model_version': '7'}, 'request_id': 'R7N5VHD1ZP3QOUSA', 'result': {'prediction': [1]}, 'timing': 16.10279083251953}
